In [1]:
import pandas as pd
import numpy as np
import os

In [8]:
df = pd.read_csv("./active_users_total.csv", encoding='cp949')

svd 협업필터링 구현

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

구현 전 평점부여(svd 적용하기 위해)

In [13]:
# 각 장소의 방문 횟수를 계산하여 인기도 기반 평점 부여
venue_count = df['Venue ID'].value_counts()
venue_count_dict = venue_count.to_dict()
df['Rating'] = df['Venue ID'].apply(lambda x: venue_count_dict[x])
# data.to_csv('C:/Users/seohe/OneDrive/바탕 화면/thesis/active_users_total.csv', index=True)

In [32]:
df[:10]

,UserID,CheckIn,LocationFrequency,ActivityTime,Cluster,Venue ID,Venue category ID,Venue category name,Latitude,Longitude,Timezone offset in minutes,UTC time,rating,Rating
0,12,309,99,21,1,4b53672bf964a520199b27e3,4bf58dd8d48988d118951735,Food & Drink Shop,40.764016,-73.829476,-240,Tue Apr 03 21:58:10 +0000 2012,22,22
1,12,309,99,21,1,4cd95a235e1b721e58b23ed9,4bf58dd8d48988d143941735,Breakfast Spot,40.703532,-73.802305,-240,Wed Apr 04 11:24:54 +0000 2012,38,38
2,12,309,99,21,1,4a2c6356f964a52057971fe3,4bf58dd8d48988d1ff941735,Miscellaneous Shop,40.762168,-73.831571,-240,Wed Apr 04 22:46:01 +0000 2012,5,5
3,12,309,99,21,1,4b53672bf964a520199b27e3,4bf58dd8d48988d118951735,Food & Drink Shop,40.763963,-73.829124,-240,Wed Apr 04 23:43:16 +0000 2012,22,22
4,12,309,99,21,1,4cd95a235e1b721e58b23ed9,4bf58dd8d48988d143941735,Breakfast Spot,40.703532,-73.802305,-240,Thu Apr 05 11:04:28 +0000 2012,38,38
5,12,309,99,21,1,4b5fbcf0f964a520b2ca29e3,4bf58dd8d48988d10f951735,Drugstore / Pharmacy,40.605899,-73.984538,-240,Fri Apr 06 15:16:32 +0000 2012,34,34
6,12,309,99,21,1,4adcda72f964a520504521e3,4bf58dd8d48988d142941735,Asian Restaurant,40.599584,-73.989795,-240,Sat Apr 07 20:46:38 +0000 2012,4,4
7,12,309,99,21,1,4cbf3eb47dc9a093adb937f5,4bf58dd8d48988d16a941735,Bakery,40.609041,-73.999740,-240,Sun Apr 08 15:40:16 +0000 2012,1,1
8,12,309,99,21,1,4bb7cf5b3db7b7135499209a,4bf58dd8d48988d142941735,Asian Restaurant,40.763334,-73.809310,-240,Sun Apr 08 17:53:57 +0000 2012,1,1
9,12,309,99,21,1,4b53672bf964a520199b27e3,4bf58dd8d48988d118951735,Food & Drink Shop,40.764016,-73.829476,-240,Sun Apr 08 21:13:39 +0000 2012,22,22


In [11]:
# Surprise 데이터셋으로 변환
from surprise import Dataset
from surprise import Reader

In [14]:
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(1, 5))
data = Dataset.load_from_df(df[['UserID', 'Venue ID', 'Rating']], reader=reader)

협업 구현

In [15]:
# 데이터 형식 지정
reader = Reader(rating_scale=(1, 5))

In [25]:
# 데이터프레임을 데이터셋으로 변환
data = Dataset.load_from_df(df[['UserID', 'Venue ID', 'Rating']], reader)

In [26]:
# 학습용 데이터와 테스트용 데이터로 분할
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [33]:
# SVD 모델 학습
model = SVD(n_factors=100, n_epochs=20, random_state=42)
model.fit(trainset)

In [24]:
from surprise import accuracy
from sklearn.metrics import precision_recall_fscore_support

# 학습 데이터로 모델 학습
model.fit(trainset)

# 테스트 데이터로 모델 평가
predictions = model.test(testset)

# 예측값과 실제값을 리스트로 저장
pred = []
true = []
for uid, iid, r_ui, est, _ in predictions:
    pred.append(est)
    true.append(r_ui)

# NaN 값 제거
pred = np.nan_to_num(pred)

# 정밀도, 재현율, F1-score 계산
precision, recall, f1_score, _ = precision_recall_fscore_support(true, np.around(pred), average='weighted')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)

Precision:  0.13449290631160404
Recall:  0.12433238107067444
F1-score:  0.11155277327643778


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
